In [ ]:
# Configure the necessary Python module imports for dashboard components
from dash import Dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson.objectid import ObjectId

import Project_One as AAC

###########################
# Data Manipulation / Model
###########################

# hard-wired database user and pass
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AAC.AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the "_id" column and that is going to have an 
# invlaid object type of "ObjectID" - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True, errors="ignore")

## Debug
# print(len(df.to_dict(orient="records")))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# logo img path
image_filename = "./Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read())


app.layout = html.Div([
    # Logo
    dcc.Link(
        html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()),
                 alt="Grazioso Logo", height="200", width="200"),
                href="snhu.edu"),

    # Title and Header
    html.Center(html.B(html.H1("CS-340 Dashboard"))),
    html.H1("Alec Novak - SNHU"),
    html.Hr(),

    # Search bar that passes input value for query
    dcc.Input(id="search_field", type="text", placeholder="Search animals by attribute:"),

    # Search button to send search query
    html.Hr(),

    # Search filter for query
    dcc.Dropdown(
        options=[
            {"label": "None", "value": ""},
            {"label": "id", "value": "animal_id"},
            {"label": "type", "value": "animal_type"},
            {"label": "breed", "value": "breed"},
            {"label": "name", "value": "name"},
        ],
        value="",
        id="search_filter",
        style={"width": "25%", "height": "50px"},
    ),

# radio items that pass filter value to callback
#     dcc.RadioItems(
#         id="animal_filter",
#         options=[{"label": i, "value": i} for i in ["All Animals", "Water", "Wilderness/Mountains", "Disaster/Tracking"]],
# value="All Animals"),
    html.Hr(),
    dash_table.DataTable(id="datatable-id",
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],

# interactivity to make data more manageable
        data=df.to_dict("records"),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className="row",
         style={"display" : "flex"},
             children=[
        html.Div(
            id="graph-id",
            className="col s12 m6",

            ),
        html.Div(
            id="map-id",
            className="col s12 m6",
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# search bar callback
@app.callback(
        Output("datatable-id", "data"),
        [Input("search_field", "value"),
         Input("search_filter", "value")],
)

def get_search_bar_query(search_value, filter):
    # load data
    df = pd.DataFrame.from_records(db.read({}))

    if (search_value):
        df = pd.DataFrame.from_records(db.read({
            filter : {"$regex": search_value, "$options": "i"}
        }))


    # if no data is returned, return full dataset
    if (len(df) == 0):
        print("No data found, returning full dataset")
        df = pd.DataFrame.from_records(db.read({}))
    
    # drop _id column
    df.drop(columns=["_id"],inplace=True)
    data = df.to_dict("records")
    
    return (data)

# # filtering callback
# @app.callback(
#     Output("datatable-id", "data"),
#     Input("animal_filter", "value")
# )

# def animal_filter(filter):
#     # load data
#     df = pd.DataFrame.from_records(db.read({}))
    
#     # query data dependent on filter
#     #reset filter
#     if (filter == "All Animals"):
#         df = pd.DataFrame.from_records(db.read({}))
        
#     # filter animals that meet requirements for water rescue
#     elif (filter == "Water"):
#         df = pd.DataFrame.from_records(db.read({"breed" : {
#             "$in":
#                 ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
#                 "sex_upon_outcome": "Intact Female",
#                 "age_upon_outcome_in_weeks": {"$gte": 26},
#                 "age_upon_outcome_in_weeks": {"$lte": 156}}))
        
#     # filter animals that meet requirements for wilderness and mountain rescue 
#     elif (filter == "Wilderness/Mountains"):
#         df = pd.DataFrame.from_records(db.read({"breed" : {
#             "$in":
#                 ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
#                 "sex_upon_outcome": "Intact Male",
#                 "age_upon_outcome_in_weeks": {"$gte": 26},
#                 "age_upon_outcome_in_weeks": {"$lte": 156}}))
        
#     #filter animals that meet requirements for disaster and tracking rescue
#     elif (filter == "Disaster/Tracking"):
#         df = pd.DataFrame.from_records(db.read({"breed" : {
#             "$in":
#                 ["Doberman Pinscher", "Alaskan Malamute", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
#                 "sex_upon_outcome": "Intact Male",
#                 "age_upon_outcome_in_weeks": {"$gte": 20},
#                 "age_upon_outcome_in_weeks": {"$lte": 300}}))
        
#     # if a different value than expected, reset filter
#     else:
#         df = pd.DataFrame.from_records(db.read({}), nrows=100)
        
#     # drop _id column
#     df.drop(columns=["_id"],inplace=True)
#     data = df.to_dict("records")
    
#     return (data)

# Display the breeds of animal based on quantity represented in
# the data table, connected to filter
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_viewport_data"),
    Input("search_filter", "value"),
    Input("search_filter", "options")])

def update_graphs(viewData, filter, options):
    
    df = pd.DataFrame.from_dict(viewData)

    # default label
    label = "breed"

    # Get filter"s label for title
    for option in options:
        if option["value"] == filter:
            label = option["label"]
            break
    
    # Query data dependent on filter, No filter -> categorize breed by default
    if (not filter or filter == "" or filter == "None"):
        filter = "breed"
        fig=px.pie(df, names=df[filter], title=f"All animals")
        fig.update_traces(hoverinfo='none', hovertemplate=None)
    else:
        fig=px.pie(df, names=df[filter], title=f"All animals by animal {label}")
        fig.update_traces(hoverinfo='none', hovertemplate=None)

    # drop _id column
    df.drop(columns=["_id"], inplace=True, errors="ignore")

    return dcc.Graph(
        figure=fig
    ) 
    
# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    return [{
        "if": { "column_id": i },
        "background_color": "#D2F3FF"
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={"width": "1000px", "height": "500px"}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app.run(mode="inline", debug=True, port=8050)